In [2]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

import pandas as pd

import os
import re

from dotenv import load_dotenv
load_dotenv()

def get_file_names(folder_path, format=".pdf"):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.
    """
    import os
    
    try:
        all_files = os.listdir(folder_path)
        pdf_files = [file.replace(format,"") for file in all_files if file.lower().endswith(format)]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []
    

In [3]:
from loader import load_process_split_doc, load_process_split_doc_law

# tax_etc 파일들 정의의
doc_options = [
    {
        "filename":"연말정산_신고안내", 
        "page_ranges": [(17, 426)],
        "table_pages_range": "17-425",
        "keyword": "", # "연말정산, 연말정산 신고, 연말정산 신고 안내",
        "effective_year": 2024,
        "replace_string": [
            {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"},
            {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"  },
            {"target_str": r"II. 2024년 귀속 연말정산 주요 일정\n\d+|"},
            {"target_str": r"III\. 원천징수의무자의 연말정산 중점 확인사항\n\d+|"},
            {"target_str": r"원천징수의무자를 위한 \n2024년 연말정산 신고안내\n\d+|"},
            {"target_str": r"Ⅰ\. 근로소득\n\d+|"},
            {"target_str": r"II\. 근로소득 원천징수 및 연말정산\n\d+|"},
            {"target_str": r"III\. 근로소득공제, 인적공제, 연금보험료공제\n\d+|"},
            {"target_str": r"IV\. 특별소득공제(소법 §52)\n\d+|"},
            {"target_str": r"V\. 그 밖의 소득공제(조특법)\n\d+|"},
            {"target_str": r"VI\. 세액감면(공제) 및 농어촌특별세\n\d+|"},
            {"target_str": r"I\. 2024년 귀속 연말정산 종합사례\n\d+|"},
            {"target_str": r"II\. 근로소득 원천징수영수증(지급명세서) 작성요령\n\d+|"},
            {"target_str": r"IV\. 수정 원천징수이행상황신고서 작성사례(과다공제)\n\d+|"},
            {"target_str": r"VI\. 홈택스를 이용한 연말정산 신고(근로소득 지급명세서 제출)\n\d+|"},
            {"target_str": r"I\. 사업소득 연말정산\n\d+|"},
            {"target_str": r"II\. 연금소득 연말정산\n\d+|"},
            {"target_str": r"I\. 종교인소득이란?\n\d+|"},
            {"target_str": r"IV\. 종교인소득(기타소득)에 대한 연말정산\n\d+|"},
            {"target_str": r"부록1\. 연말정산 관련 서비스\n\d+|"},
            {"target_str": r"부록2\. 연말정산간소화 서비스\n\d+|"},
            {"target_str": r"부록5\. 연말정산 주요 용어 설명\n\d+|"},
            {"target_str": r"부록6\. 소득·세액공제신고서 첨부서류\n\d+|"},
            {"target_str": r"\bNaN\b"},
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])","replace_str": r"\1\2"},
            {"target_str": r"\s+", "replace_str": r" "},
        ]
    },
    {
        "filename":"연말정산_주택자금·월세액_공제의이해", 
        "page_ranges": [(9, 12), (15, 24), (27, 28), (31, 39), (43, 70)],
        "table_pages_range": "9-12,15-24,27-28,31-39,43-70",
        "keyword": "", # "연말정산, 연말정산_주택자금, 월세액_공제의이해, 연말정산_주택자금·월세액_공제의이해",
        "effective_year": 2024,
        "replace_string": [
            {"target_str": (r"연말정산 주택자금･월세액 공제의 이해\n\d+|" r"\bNaN\b"),  },
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
            {"target_str": r"\s+", "replace_str": " " },
        ]
    },
    {
        "filename":"2024_핵심_개정세법", 
        "page_ranges": [(19, 44), (47, 71), (75, 161)],
        "table_pages_range": "19-44,47-71,75-161",
        "keyword": "", # "적용시기 2025. 1. 1., 개정, 종전전",
        "effective_year": 2025,
        "replace_string": [
            {"target_str": (r"2\n0\n2\n5\n\s*달\n라\n지\n는\n\s*세\n금\n제\n도|"  
                r"\n2\n0\n2\n4\n\s*세\n목\n별\n\s*핵\n심\n\s*개\n정\n세\n법|"
                r"\n2\n0\n2\n4\n\s*개\n정\n세\n법\n\s*종\n전\n-\n개\n정\n사\n항\n\s*비\n교\n|"
                r"\s*3\s*❚국민･기업\s*납세자용\s*|"
                r"\s*2\s*0\s*2\s*4\s|"
                r"\s한국세무사회\s|" 
                r"\n7\n❚국민･기업 납세자용|"
                r"\n71\n❚상세본|"),  
            },
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
            {"target_str": r"\s+", "replace_str": " " },
        ]
    },
    {
        "filename":"주요_공제_항목별_계산사례", 
        "page_ranges": [(1,1), (4, 21)],
        "table_pages_range": "1,4-21",
        "keyword": "", # "공제, 계산, 계산사례, 세액공제, 소득공제, 기본공제, 인적공제, 추가공제제",
        "effective_year": 2025,
        "replace_string": [
            {"target_str": r"\bNaN\b"},
            {"target_str": r"\s+", "replace_str": " " },
        ]
    }
    
]

# vector store 에 넣을 데이터 document 모으기 -> all_documents
all_documents = []
# load - 세법 
law_files = get_file_names("data/tax_law")
for file in law_files:
    all_documents.extend(load_process_split_doc_law(file))
# load - 참고자료료
for doc in doc_options:
    all_documents.extend(load_process_split_doc(**doc))
    

c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 15 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 37 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 223 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 1 of document data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3

In [4]:
len(all_documents)

6241

In [5]:
COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

def set_vector_store(documents):
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

    return Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )

In [6]:
# 📌 vector store 생성
vector_store = set_vector_store(all_documents)


In [7]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 10}
)

In [8]:
# Prompt Template 생성
messages = [
        ("ai", """
        당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 저장된 세법 조항 데이터를 기반으로 사용자 질문에 답변하세요.

        - 모든 답변은 학습된 세법 데이터 내에서만 유효한 정보를 바탕으로 작성하세요. 데이터에 없는 내용은 추측하거나 임의로 생성하지 마세요.
        - 질문에 명확한 답변이 없거나 데이터 내에서 찾을 수 없는 경우, 정직하게 "잘 모르겠습니다."라고 말하고, 새로운 질문을 유도하세요.
        - 질문이 포함된 조항뿐 아니라, 필요 시 서로 연관된 다른 조항도 참고하여 답변의 정확성과 완성도를 높이세요.
        - 사용자가 이해하기 쉽게 답변을 구성하며, 중요한 키워드나 법 조항은 명확히 표시하세요.
        - 세법과 관련된 복잡한 질문에 대해서는 관련 조항 번호와 요약된 내용을 포함하여 답변을 제공하세요.
        - 최대한 학습하고있는 내용이 질문과 유사하다고 판단되면 그 답을 제공하세요.
        
        추가 규칙:
        답변은 간결하고 명료하게 작성하되, 필요한 경우 관련 조항의 전문을 추가적으로 인용하세요.
        세법 용어를 사용자 친화적으로 설명하여 비전문가도 쉽게 이해할 수 있도록 하세요.
        질문을 완전히 이해하기 어렵거나 모호할 경우, 사용자가 구체적으로 질문을 다시 작성할 수 있도록 유도하는 후속 질문을 하세요.
        
        답변 할 때, 개행문자 두개 이상 연속으로 절대 사용하지 마세요.

	답변 후, 사용자에게 필요할 것 같은 정보를 바탕으로 두 가지 후속 질문을 제안하세요. 각 질문의 앞뒤에 한 줄씩 띄어쓰기를 하세요. 이 질문은 원래 주제와 관련된 내용이어야 합니다.
	특정 법률 조항이나 제도가 언급될 경우, 근거가 되는 세법 조문, 시행령, 또는 관련 자료를 명시합니다.
        모든 답변은 사용자에게 법적 조언이 아닌 정보 제공 목적으로 작성된 것임을 명확히 합니다. 
        
	{context}")"""
        ),
        ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser


In [9]:
# chain.invoke("개별소비세법이 뭐야?")
chain.invoke("개별소비세법 제1조가 뭐야")
# chain.invoke("교통_에너지_환경세법 뭐야?")
# chain.invoke("연봉 6000에 근로소득을 가지고 있는 근로자인데 연말정산하는 방법 알려줘")

'죄송하지만 현재 제공된 문서에는 개별소비세법 제1조에 대한 구체적인 내용이 포함되어 있지 않습니다. 일반적으로, 세법의 제1조는 법의 목적이나 적용 범위에 대해 설명하는 경우가 많습니다. \n\n혹시 다른 관련 조항이나 주제에 대해 궁금한 점이 있으면 말씀해 주세요. 추가로 도움 드릴 수 있는 부분이 있다면 최대한 도와드리겠습니다.\n\n혹시 다음과 같은 질문에 관심이 있으신가요?\n\n1. 개별소비세법의 과세 물품에는 어떤 것들이 포함되나요?\n2. 개별소비세의 과세 표준은 어떻게 결정되나요?'